In [5]:
from datasets import Dataset
import pandas as pd
import evaluate
import numpy as np
from transformers import EarlyStoppingCallback,AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed
import os
from sklearn.model_selection import train_test_split
from scipy.special import softmax
import argparse
import logging
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from datasets import Dataset
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from textstat.textstat import textstatistics
import pandas as pd
import language_tool_python


In [6]:
df= pd.read_json('datasets/subtaskA_train_monolingual.jsonl', lines=True)
df=df.sample(10000)
print(df['label'].value_counts())

label
0    5309
1    4691
Name: count, dtype: int64


In [7]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Initialize the grammar checker
tool = language_tool_python.LanguageTool('en-US')

def extract_text_features(text):
    # Tokenize the text
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    word_count = len(words)
    sentence_count = len(sentences)

    # Lexical diversity
    unique_words = set(words)
    lexical_diversity = len(unique_words) / word_count

    # Average sentence length
    avg_sentence_length = word_count / sentence_count

    # Average word length
    avg_word_length = sum(len(word) for word in words) / word_count

    # Lexical density
    tags = nltk.pos_tag(words)
    content_words = [word for word, tag in tags if tag.startswith(('N', 'V', 'J', 'R'))]
    lexical_density = len(content_words) / word_count

    # Readability scores
    flesch_reading_ease = textstatistics().flesch_reading_ease(text)
    fog_index = textstatistics().gunning_fog(text)

    # Grammatical errors
    matches = tool.check(text)
    grammatical_errors = len(matches)

    # Create a DataFrame
    return pd.Series({
        'word_count': word_count,
        'sentence_count': sentence_count,
        'lexical_diversity': lexical_diversity,
        'avg_sentence_length': avg_sentence_length,
        'avg_word_length': avg_word_length,
        'lexical_density': lexical_density,
        'flesch_reading_ease': flesch_reading_ease,
        'gunning_fog_index': fog_index,
        'grammatical_errors': grammatical_errors
    })

    return df

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ghiki\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ghiki\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [8]:
from tqdm import tqdm
tqdm.pandas()
features_df = df['text'].progress_apply(extract_text_features)
features_df

100%|██████████| 10000/10000 [24:40<00:00,  6.76it/s] 


,word_count,sentence_count,lexical_diversity,avg_sentence_length,avg_word_length,lexical_density,flesch_reading_ease,gunning_fog_index,grammatical_errors
112437,285.0,11.0,0.543860,25.909091,5.164912,0.617544,35.88,12.34,6.0
33312,298.0,11.0,0.503356,27.090909,4.986577,0.550336,37.84,15.93,2.0
43548,238.0,13.0,0.466387,18.307692,5.172269,0.567227,37.50,12.41,1.0
90838,229.0,26.0,0.576419,8.807692,3.864629,0.510917,71.31,6.41,5.0
39065,273.0,9.0,0.527473,30.333333,4.761905,0.538462,36.42,15.56,15.0
...,...,...,...,...,...,...,...,...,...
58200,653.0,21.0,0.437979,31.095238,4.304747,0.493109,54.66,13.24,20.0
60295,1025.0,55.0,0.318049,18.636364,3.715122,0.480976,88.26,7.54,14.0
110977,321.0,7.0,0.526480,45.857143,4.750779,0.504673,21.50,22.51,7.0
25456,190.0,5.0,0.447368,38.000000,5.115789,0.515789,27.08,18.45,2.0


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Prepare your data
X = features_df  # Your extracted features
y = df['label']  # Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Get feature importances
importances = clf.feature_importances_
feature_names = X.columns

# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]

# Print the feature rankings
print("Feature ranking:")
for f in range(X.shape[1]):
    print(f"{f + 1}. feature {feature_names[indices[f]]} ({importances[indices[f]]})")

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import precision_score, recall_score, roc_auc_score, confusion_matrix
import numpy as np

def getMetrics(predicted_labels, true_labels):
    # Ensure the labels are numpy arrays
    predicted_labels = np.array(predicted_labels)
    true_labels = np.array(true_labels)

    # Compute metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    f1 = precision_recall_fscore_support(true_labels, predicted_labels, average='macro')[2]
    precision = precision_score(true_labels, predicted_labels, average='binary')
    recall = recall_score(true_labels, predicted_labels, average='binary')
    auc = roc_auc_score(true_labels, predicted_labels)
    cm = confusion_matrix(true_labels, predicted_labels)

    # Create a dictionary of metrics
    metrics = {
        'f1': f1,
        'confusion_matrix': cm.tolist(),
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'auc': auc,
    }

    return metrics

# Assuming clf is your trained model and X_test, y_test are your test features and labels
predicted_labels = clf.predict(X_test)
metrics = getMetrics(predicted_labels, y_test)

# Print the metrics
for metric, value in metrics.items():
    print(f"{metric}: {value}")


Feature ranking:
1. feature lexical_diversity (0.1959888298175121)
2. feature grammatical_errors (0.18588589402068853)
3. feature word_count (0.14293442095584902)
4. feature avg_word_length (0.10139951225306561)
5. feature avg_sentence_length (0.09058214449399676)
6. feature flesch_reading_ease (0.0723916598476359)
7. feature sentence_count (0.07148867120696319)
8. feature lexical_density (0.07134239322180339)
9. feature gunning_fog_index (0.06798647418248556)
f1: 0.8415728446802802
confusion_matrix: [[919, 145], [170, 766]]
accuracy: 0.8425
precision: 0.8408342480790341
recall: 0.8183760683760684
auc: 0.8410489364436733


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [250,300,350],  # Number of trees in the random forest
    'max_features': ['sqrt'],  # Number of features to consider at every split
    'max_depth': [None],  # Maximum number of levels in tree
    'min_samples_split': [3,5,7,9],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1],  # Minimum number of samples required at each leaf node
    'bootstrap': [True]  # Method of selecting samples for training each tree
}

# Create a base model
rf = RandomForestClassifier(random_state=42)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='accuracy')
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)

best_grid = grid_search.best_estimator_

# Now you can use best_grid to make predictions and evaluate it
predicted_labels = best_grid.predict(X_test)
metrics = getMetrics(predicted_labels, y_test)

# Print the metrics
for metric, value in metrics.items():
    print(f"{metric}: {value}")


Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best Parameters: {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 7, 'n_estimators': 350}
f1: 0.8361024972750783
confusion_matrix: [[911, 153], [173, 763]]
accuracy: 0.837
precision: 0.8329694323144105
recall: 0.8151709401709402
auc: 0.8356869738448686


In [11]:
test_df= pd.read_json('datasets/subtaskA_dev_monolingual.jsonl', lines=True)

test_features_df = test_df['text'].progress_apply(extract_text_features)
print(test_features_df.shape)

test_X = test_features_df  # Your extracted features
test_y = test_df['label']  # Labels

predicted_labels = best_grid.predict(test_X)
getMetrics(predicted_labels, test_y)

100%|██████████| 5000/5000 [09:35<00:00,  8.69it/s]  

(5000, 9)


{'f1': 0.47029538665781623,
 'confusion_matrix': [[2078, 422], [1972, 528]],
 'accuracy': 0.5212,
 'precision': 0.5557894736842105,
 'recall': 0.2112,
 'auc': 0.5211999999999999}

In [12]:
print('ola')

ola


In [13]:
import pickle
with open('filename.pkl', 'wb') as file:
    pickle.dump(best_grid, file)
